In [6]:
from pymongo import MongoClient
import requests

client = MongoClient('ec2-52-211-76-129.eu-west-1.compute.amazonaws.com',27017)
db = client.wax_dfg
deputes_db = db.deputes

deputes_enmandat_from_api = requests.get('https://www.nosdeputes.fr/deputes/enmandat/json')
deputes_enmandat_json = deputes_enmandat_from_api.json()
deputes_db.drop()
for depute in deputes_enmandat_json['deputes']:   
        url_depute_json = depute['depute']['url_nosdeputes_api']
        depute_from_api = requests.get(url_depute_json)
        depute_json = depute_from_api.json()
        departement = db.departement.find({'DEP':depute_json['depute']['num_circo']})
        depute_json['depute']['region'] ={'num': departement[0]['REGION']['REGION'], 'nom':departement[0]['REGION']['NCCENR']}
        depute_json['depute']['date_naissance'] = int(depute_json['depute']['date_naissance'].replace('-',''))
        deputes_db.insert_one(depute_json['depute'])